In [3]:
import os
import sys
sys.path.append('..')

import openai
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.text_splitter import TokenTextSplitter, CharacterTextSplitter

from azure.core.credentials import AzureKeyCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    ScoringProfile,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    TextWeights,
)

from scripts.table_and_text_parser import start_blob_client, extract_blob_paths, parse_pdfs, page_text_and_tables

from dotenv import load_dotenv
load_dotenv() # load environment variables from .env

True

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
openai.api_type="azure"
# openai.api_version="2023-09-15-preview"
openai.api_version="2023-05-15"
openai.api_base="https://use-gaa-openai-test1.openai.azure.com/"
openai.api_key=os.getenv('OPENAI_API_KEY')

In [6]:
container_client = start_blob_client(
    azure_storage_connection_string=os.environ['AZURE_STORAGE_CONNECTION_STRING'],
    azure_storage_container_name=os.environ['AZURE_STORAGE_CONTAINER_NAME']
    )

In [7]:
list_of_blob_paths = extract_blob_paths(container_client)
result_dicts = parse_pdfs(list_of_blob_paths)

In [8]:
# print('Number of reports:', len(result_dicts))
# print('\nReport names:')
# for result_dict in result_dicts:
    # print(result_dict.get('tables'))
    # print(result_dict.get('report_name'))

In [9]:
paged_text_and_tables = page_text_and_tables(result_dicts)

In [34]:
def cleanup_whitespace(s:str):
    return re.sub("\s+", " ", s)

def cleanup_multiple_underlines(s:str):
    return re.sub(r"_{2,}", " ", s)

def cleanup_escapechar(s:str):
    return re.sub("\\'", "", s)

def cleanup_copyrights_characters(s:str):
    return re.sub('Copyright ©.+(Manulife)', '', s)

def preprocess_text(text):
    text = cleanup_whitespace(text)
    text = cleanup_multiple_underlines(text)
    text = cleanup_copyrights_characters(text)
    text = cleanup_escapechar(text)
    # metadata_text = doc.metadata.get('text')
    # metadata_text = cleanup_whitespace(doc.metadata.get('text'))
    # metadata_text = cleanup_multiple_underlines(doc.metadata.get('text'))
    # metadata_text = cleanup_copyrights_characters(doc.metadata.get('text'))
    # metadata_text = cleanup_escapechar(doc.metadata.get('text'))
    return text

In [60]:
def convert_pages_to_table_docs(paged_text_and_tables, metadata_page_span=1):

    lang_doc_tables = []
    for i, report in enumerate(paged_text_and_tables):
        num_pages = max(list(report.keys()))
        for page_num, tables_and_text in report.items():
            for table in tables_and_text.get('tables'):
                # print(table, '\n')
                # continue
                # print('Length of original text:', len(tables_and_text.get('text')))
                # print(tables_and_text.get('text'), '\n')
                # tables_and_text.get('text')[:] = \
                #     [text for text in tables_and_text.get('text') if text not in table.values]
                # print('Length of deduplicated text:', len(tables_and_text.get('text')))
                # print(tables_and_text.get('text'))
                # return
                if metadata_page_span == 1:
                    # tables_and_text.get('text')[:] = \
                    #     [text for text in tables_and_text.get('text') if text not in table.values]
                    metadata = preprocess_text(' '.join(tables_and_text.get('text')))
                    lang_doc_tables.append(
                        Document(
                            page_content=table.to_string(),
                            # page_content=str(table),
                            metadata={
                                'text': metadata, 
                                'page_num': page_num,
                                }
                            )
                        )
                # 'report_name': report.get('report_name')}))
                #     metadata = ''.join(report[page_num-1].get('text')) \
                #                + ''.join(report[page_num].get('text')) \
                #                + ''.join(report[page_num+1].get('text'))
                #     # print('Metadata for page:', page_num, '\n', metadata)
                #     # if metadata is None:
                #     #     metadata = ''
                # elif page_num == num_pages:
                #     metadata = ''.join(report[page_num-1].get('text')) \
                #                + ''.join(report[page_num].get('text'))
                #     # if metadata is None:
                #     #     metadata = ''
                #     # print('Metadata for page:', page_num, '\n', metadata)
                # else:
                #     metadata = ''.join(report[page_num+1].get('text')) \
                #                + ''.join(report[page_num].get('text'))
                #     # if metadata is None:
                #     #     metadata = ''
                #     # print('Metadata for page:', page_num, '\n', metadata)

    return lang_doc_tables
    
lang_doc_tables = convert_pages_to_table_docs(paged_text_and_tables)

In [61]:
# text_splitter = TokenTextSplitter(chunk_size=400, chunk_overlap=0)
text_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=0)
lang_doc_tables_chunks = text_splitter.split_documents(lang_doc_tables)

In [62]:
# from pandas.compat import StringIO #if this doesn't work try: from io import StringIO
# import StringIO

for i, chunk in enumerate(lang_doc_tables_chunks):
    # read from df.to_string() back to dataframe??
    # df = pd.read_csv(StringIO(chunk.page_content), sep='\s+')
    print('\nChunk:', i, 'Chunk length:', len(chunk.page_content), '\n', chunk.page_content)
    print('Chunk metadata:\n', chunk.metadata)
    if i == 1:
        break

print('\nTotal number of chunks:', len(lang_doc_tables_chunks))


Chunk: 0 Chunk length: 13064 
 0   Selected consolidated financial information (in Canadian $ millions, except per share amounts)  Selected consolidated financial information As at or for the three months ended Dec. 31 2023  Selected consolidated financial information As at or for the three months ended Sept. 30 2023  Selected consolidated financial information As at or for the three months ended Dec. 31 2022 (Restated)  Selected consolidated financial information For the twelve months ended Dec. 31 2023  Selected consolidated financial information For the twelve months ended Dec. 31 2022 (Restated)
1                                                                                Base earnings 1,5                                                                                         $ 971                                                                                          $ 950                                                                                                    $ 894

In [63]:
embeddings = OpenAIEmbeddings(
    deployment='text-embedding-ada-002-v2',
    openai_api_base=os.environ['OPENAI_API_BASE'],
    openai_api_type=os.environ['OPENAI_API_TYPE'],
    openai_api_key=os.environ['OPENAI_API_KEY'],
    openai_api_version=os.environ['OPENAI_API_VERSION'],
    # chunk_size = 1
    )

embedding_function=embeddings.embed_query

In [46]:
azure_search_endpoint = "https://" + os.environ['AZURE_AI_SEARCH_SERVICE_NAME'] + ".search.windows.net"
search_client = SearchIndexClient(azure_search_endpoint, AzureKeyCredential(os.environ['AZURE_AI_SEARCH_KEY']))

In [47]:
def index_docs(search_client, lang_doc_tables_chunks, overwrite_existing_index=True):
    try:
        search_client.get_index(os.environ['AZURE_AI_SEARCH_INDEX_NAME'])
    except:
        print('No existing index with name', os.environ['AZURE_AI_SEARCH_INDEX_NAME'], \
                'in search service', os.environ['AZURE_AI_SEARCH_SERVICE_NAME'])
    else:
        print('Existing index', os.environ['AZURE_AI_SEARCH_INDEX_NAME'], 'in search service', \
                os.environ['AZURE_AI_SEARCH_SERVICE_NAME'])
    finally:
        if overwrite_existing_index:
            print('Deleting existing index', os.environ['AZURE_AI_SEARCH_INDEX_NAME'], 'in search service', \
                    os.environ['AZURE_AI_SEARCH_SERVICE_NAME'])
            search_client.delete_index(os.environ['AZURE_AI_SEARCH_INDEX_NAME'])
            print('Creating new index', os.environ['AZURE_AI_SEARCH_INDEX_NAME'], 'in search service', \
                    os.environ['AZURE_AI_SEARCH_SERVICE_NAME'])
            acs_vector_store = AzureSearch(
                azure_search_endpoint=azure_search_endpoint,
                azure_search_key=os.environ['AZURE_AI_SEARCH_KEY'],
                index_name=os.environ['AZURE_AI_SEARCH_INDEX_NAME'],
                embedding_function=embedding_function,
                # fields=fields,
            )
            import time
            t = time.time()
            print('Pushing documents to Azure vector store...')
            acs_vector_store.add_documents(documents=lang_doc_tables_chunks)
            print(len(lang_doc_tables_chunks), 'documents successfully indexed in', \
                    round(time.time() - t), 'seconds')
        return acs_vector_store

In [48]:
acs_vector_store = index_docs(search_client, lang_doc_tables_chunks)

Existing index messiah_search in search service nlp-ai-search1
Deleting existing index messiah_search in search service nlp-ai-search1
Creating new index messiah_search in search service nlp-ai-search1
Pushing documents to Azure vector store...
136 documents successfully indexed in 18 seconds


In [58]:
# input_type = 'List of JSONs'
# input_type = 'List of DataFrames'

query = """
What are the Full Year Core Earnings for 2023?
"""

returned_chunks = acs_vector_store.similarity_search(
    query=query,
    k=3,
    search_type="similarity",
)

for i, chunk in enumerate(returned_chunks):
    print('Chunk', i, 'table', '\n', chunk.page_content)
    print('Metadata', chunk.metadata, '\n')
    if i == 1:
        break

input = []
for chunk in returned_chunks:
    input.append(chunk.page_content)

Chunk 0 table 
 0                                                             Quarterly Results 4Q23  Quarterly Results 3Q23  Quarterly Results 2Q23  Quarterly Results 1Q23  Quarterly Results 4Q22  Full Year Results 2023  Full Year Results 2022
0                                              Core earnings                 $ 1,773                 $ 1,743                 $ 1,637                 $ 1,531                 $ 1,543                 $ 6,684                 $ 5,801
1                            Less: Preferred share dividends                    (99)                    (54)                    (98)                    (52)                    (97)                   (303)                   (260)
2             Core earnings available to common shareholders                   1,674                   1,689                   1,539                   1,479                   1,446                   6,381                   5,541
3                                          CER adjustment(1)        

In [51]:
prompt = f"""
Answer the QUESTION enclosed in the dollar signs (i.e, $) from the data enclosed in triple backticks (i.e., ```).
Do not answer from memory. If you do not know an answer, just say I do not know.

QUESTION: 
$
{query}
$

```
{input}
```
"""

message_text = [{"role":"system","content":"You are an AI assistant that helps people find information."},
{"role": "user","content": prompt}]

completion = openai.ChatCompletion.create(
  engine="gpt-4-32k", # model = "deployment_name" # try gpt-4
  messages = message_text,
  temperature=0.7, # 0.7
  max_tokens=800,
  top_p=0.95
)

# print(prompt)
completion.get('choices')[0].get('message').get('content')

'The Full Year Core Earnings for 2023 are $ 6,684.'